# First Step: As always, make sure we can read the data!
I usually just use the top block to make sure I can actually read the data and install all the libraries beforehand since it can be a bit of a mess/hard to read doing this stuff later into the notebook! Here I'm just making sure that I can actually access the dataset for the most part (as well as seeing if there are any problems with the current Python environment/kernel on the side...)

In [2]:
# Install some useful packages for data analysis and visualization
!pip install pandas --break-system-packages
!pip install matplotlib --break-system-packages
!pip install seaborn --break-system-packages
!pip install scikit-learn --break-system-packages
!pip install tensorflow --break-system-packages
!pip install keras --break-system-packages


from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

# Load the dataset
data = pd.read_csv('FoodX.csv')

# Display the first few rows of the dataset
data.head()

,Year,Major,University,Time,Order
0,Year 2,Physics,Indiana State University,12,Fried Catfish Basket
1,Year 3,Chemistry,Ball State University,14,Sugar Cream Pie
2,Year 3,Chemistry,Butler University,12,Indiana Pork Chili
3,Year 2,Biology,Indiana State University,11,Fried Catfish Basket
4,Year 3,Business Administration,Butler University,12,Indiana Corn on the Cob (brushed with garlic b...


# Next step: Choosing a model

I'm going to try and go with a Random Forest or Decision Tree approach first since this just seems like a very easy classification problem and those are like the best (not to mention easiest) ways of handling that kind of dataset. I'm just going to use scikit-learn since it's the quickest way to get these two models up and running :)

# Random Forest Implementation

In [3]:
# Data preprocessing
# Encode categorical variables using LabelEncoder
label_encoder = LabelEncoder()
data['Year'] = label_encoder.fit_transform(data['Year'])
data['Major'] = label_encoder.fit_transform(data['Major'])
data['University'] = label_encoder.fit_transform(data['University'])
data['Time'] = label_encoder.fit_transform(data['Time'])
data['Order'] = label_encoder.fit_transform(data['Order'])

# Split the data into features (X) and target (y)
X = data.drop('Time', axis=1)
y = data['Time']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

# Create and train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions on new data
y_pred = rf_model.predict(X_test)

# Evaluate the model (you can use different metrics depending on your task)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.504


In [4]:
# Define a parameter grid for Grid Search
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, scoring='accuracy', cv=5)
grid_search.fit(X_train, y_train)

# Get the best parameters and estimator
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

# Evaluate the best model on the test data
accuracy = best_estimator.score(X_test, y_test)
print(f"Best Model Parameters: {best_params}")
print(f"Accuracy on test data with best model: {accuracy:.2f}")

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


Best Model Parameters: {'criterion': 'gini', 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10}
Accuracy on test data with best model: 0.50


# Decision Tree Implementation

Let's see how it fairs against the Random Forest implementation

In [5]:
# Create a Decision Tree model and fit it to the training data
decision_tree = DecisionTreeClassifier(random_state=42)
decision_tree.fit(X_train, y_train)

# Evaluate the model's performance on the test data
accuracy = decision_tree.score(X_test, y_test)
print(f"Accuracy on test data: {accuracy:.2f}")

Accuracy on test data: 0.52


In [6]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

# Create a Decision Tree classifier
decision_tree = DecisionTreeClassifier(random_state=42)

# Define a parameter grid for Grid Search
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(estimator=decision_tree, param_grid=param_grid, scoring='accuracy', cv=5)
grid_search.fit(X_train, y_train)

# Get the best parameters and estimator
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

# Evaluate the best model on the test data
accuracy = best_estimator.score(X_test, y_test)
print(f"Best Model Parameters: {best_params}")
print(f"Accuracy on test data with best model: {accuracy:.2f}")

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


Best Model Parameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2}
Accuracy on test data with best model: 0.52


In [7]:
# Data preprocessing
# Encode categorical variables using LabelEncoder
label_encoder = LabelEncoder()
data['Year'] = label_encoder.fit_transform(data['Year'])
data['Major'] = label_encoder.fit_transform(data['Major'])
data['University'] = label_encoder.fit_transform(data['University'])
data['Time'] = label_encoder.fit_transform(data['Time'])
data['Order'] = label_encoder.fit_transform(data['Order'])

# Split the data into features (X) and target (y)
X = data.drop('Time', axis=1)
y = data['Time']

# One-hot encode the target variable (time of the food orders)
y = to_categorical(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a simple feedforward neural network
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model to the training data
model.fit(X_train, y_train, epochs=2000, batch_size=1024, verbose=0)

# Evaluate the model on the test data
accuracy = model.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy on test data with neural network: {accuracy:.2f}")


Accuracy on test data with neural network: 0.46


# Conclusions

Out of the all of the different models I tested, that being the Decision Trees, Random Forest and a simple Neural Network implementation, the Decision Tree ended up being the most performant while being slightly less accurate, but within margin of error considering all of these models plateued at around the same accuracy (~50%).

I think the dataset could be improved to include more columns since right now only a few colleges and majors are really represented with a pretty evenly distributed order time window, leading to a very generalized viewpoint of who is ordering what time that isn't very correlated with other user stats.

Finding what type of person is ordering at what time is also very hard to do just because the mass amount of people that order at the same time and the lack of extremities. This may just not be a good thing to look for.